# Graficos de la parte Analisis

## Vamos a exportar los datos de omnet.

1. generar un archivo .csv con datos en omnet
3. trabajar esa data en una notebook con jupyter/jupyterlab o colab

In [ ]:
#Configuracion inicial
%matplotlib inline
import matplotlib.pyplot as plt #librería para graficas de python
import numpy as np # extiende las librerias de matemática de python
import pandas #libreria para procesamiento de datos muy usada en la industria
import seaborn # Extiende las funcionalidades de matplotlib, muy usado en data visualization

# Con esto hacemos los gráficos más grandes, adecuados para el proyector.
##seaborn.set_context(context='talk', font_scale=1.2)


In [ ]:
#Podemos importar archivos a nuestro notebook con este snippet 
from google.colab import files  

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
#Podemos usar comandos de bash con el %
%ls

In [ ]:
## Cambiar la dirección del conjunto de datos de acuerdo a dónde lo guardaron
data = pandas.read_csv("analisis2.csv")

In [ ]:
## Veamos los datos en forma de tabla, e identificamos la fila y columna en donde estan mis datos de interés
data

## Procesamiento de datos

Limpiamos los datos para poder utilizarlos

In [ ]:
# Esto nos dice las columnas de nuestra tabla
# data.columns

#almacenamos los parametros de intervalos en una lista

aux = data.loc[data['attrname'].isin(['Network.nodeTx.gen.generationInterval'])]
aux = aux.iloc[:,5]
aux.reset_index(drop=True, inplace=True)

interval = []

for i in aux:
  interval.append(i)

interval.sort(reverse=True)

#creamos un dataframe para los vectores

vectors = data.loc[data['type'].isin(['vector'])]
vectors = vectors.drop(columns=[vectors.columns[1],
                                vectors.columns[4],
                                vectors.columns[5]])


#almacenamos las iteraciones en una lista de sub-dataframes

dataframes = []
for run, group in vectors.groupby('run'):
  dataframes.append(group)

for j, df in enumerate(dataframes):
  df['run'] = (df.iat[0, 0])[8]
  df.reset_index(drop=True, inplace=True)


print("Numero de iteraciones a analizar: ", len(dataframes))


# iterations  0, 1, 2, 3, 4 = 0.1, 0.2, 0.3, 0.5, 0.1

dataframes[0]

In [ ]:
def parse_vector(df, index):
  time = df['vectime'].loc[[index]].tolist()
  value = df['vecvalue'].loc[[index]].tolist()
  return time, value

def parse_value(val):
  res = list(map(float, val[0].split()))
  return res

array = []

for df in dataframes:
  #Omnet guarda los valores de nuestro "vector" (lista de valores) en una celda, por lo que tengo que esxtraer estos valores
  time_sentpkt, count_sentpkt = parse_vector(df, 0)
  time_gen, buffer_gen = parse_vector(df, 1)
  time_q, buffer_q = parse_vector(df, 2)
  time_noderx, buffer_noderx = parse_vector(df, 3)
  time_rcvpkt, count_rcvpkt = parse_vector(df, 5)
  parsed_data = []
  # Los valores que sacamos estan como un solostring separados por comas por lo que tengo que convertirlos a listas para poder graficarlos
  parsed_data.append((parse_value(time_gen), parse_value(buffer_gen)))
  parsed_data.append((parse_value(time_q), parse_value(buffer_q)))
  parsed_data.append((parse_value(time_noderx), parse_value(buffer_noderx)))
  parsed_data.append((parse_value(time_sentpkt), parse_value(count_sentpkt)))
  parsed_data.append((parse_value(time_rcvpkt), parse_value(count_rcvpkt)))
  if 6 in df.index:
    time_drops, count_drops = parse_vector(df, 6)
    parsed_data.append((parse_value(time_drops), parse_value(count_drops)))
  else:
    parsed_data.append(([],[]))
  array.append(parsed_data)

#array[Iteracion][Vector][Time/Value]

#dataframes[0]

## Graficamos los datos

In [ ]:
# Define el numero de subplots segun el numero de iteraciones
num_subplots = len(array)

# Calcula el numero de filas y columnas para el plot
num_rows = (num_subplots + 2) // 3 
num_cols = min(num_subplots, 3)

# Crea la figura con subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 4*num_rows))

if num_subplots == 1:
    axs = np.array([axs])

# Itera el array y traza cada subplot
for n, i in enumerate(array):
    row = n // num_cols
    col = n % num_cols

    ax = axs[row, col]  # Selecciona el subplot

    ax.plot(i[0][0], i[0][1], color='tab:blue', label="Tx")
    ax.plot(i[1][0], i[1][1], color='tab:orange', label="Nx")
    ax.plot(i[2][0], i[2][1], color='tab:green', label="Rx")

    ax.set_title('Case Study 2, generationInterval: %s' % interval[n], fontsize=9)
    ax.set_xlabel("tiempo de simulacion")
    ax.set_ylabel("Cantidad de paquetes en el buffer")
    ax.legend()

# Remueve subplots vacios
if num_subplots < num_rows * num_cols:
    for i in range(num_subplots, num_rows * num_cols):
        fig.delaxes(axs.flatten()[i])

# Titulo principal del plot
plt.suptitle('Ocupacion de buffers en el sistema', fontsize=12)

# Ajustar espacio entre subplots
plt.tight_layout()

# Mostrar plot
plt.show()


In [ ]:
# Define el numero de subplots segun el numero de iteraciones
num_subplots = len(array)

# Calcula el numero de filas y columnas para el plot
num_rows = (num_subplots + 2) // 3 
num_cols = min(num_subplots, 3)

# Crea la figura con subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
fig.suptitle('Paquetes', fontsize=16)

# Itera el array y traza cada subplot
for n, i in enumerate(array):
    row = n // num_cols
    col = n % num_cols

    ax = axs[row, col]  # Selecciona el subplot

    ax.set_title('Case Study 2, generationInterval: %s' % interval[n])

    colors = ['blue', 'green', 'red']  # Colores para las bars
    
    bars = ax.bar(['Enviados', 'Recibidos', 'Dropped'], [len(i[3][1]), len(i[4][1]), len(i[5][1])], color=colors)

    ax.text(.99, .99,
        'Buffers en t=200: Tx:%i Nx:%i Rx:%i' %(i[0][1][len(i[0][1])-1],
                                                i[1][1][len(i[1][1])-1],
                                                i[2][1][len(i[2][1])-1]),
        style='italic',
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height, '%d' % int(height),
                ha='center', va='bottom')

# Remueve subplots vacios
if num_subplots < num_rows * num_cols:
    for i in range(num_subplots, num_rows * num_cols):
        fig.delaxes(axs.flatten()[i])

# Ajustar espacio entre subplots
fig.tight_layout()

# Mostrar el plot
plt.show()


# Parte diseño

## Load data

In [ ]:
#Configuracion inicial
%matplotlib inline
import matplotlib.pyplot as plt #librería para graficas de python
import numpy as np # extiende las librerias de matemática de python
import pandas #libreria para procesamiento de datos muy usada en la industria
import seaborn # Extiende las funcionalidades de matplotlib, muy usado en data visualization

# Con esto hacemos los gráficos más grandes, adecuados para el proyector.
##seaborn.set_context(context='talk', font_scale=1.2)

In [ ]:
#Podemos importar archivos a nuestro notebook con este snippet 
from google.colab import files  

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
%ls

In [ ]:
data = pandas.read_csv("test2.csv")

data     #uncomment to show data

## Procesamiento de datos

In [ ]:

#almacenamos los parametros de intervalos en una lista

aux = data.loc[data['attrname'].isin(['Network.nodeTx.gen.generationInterval'])]
aux = aux.iloc[:,5]
aux.reset_index(drop=True, inplace=True)

interval = []

for i in aux:
  interval.append(i)

interval.sort(reverse=True)

#creamos un dataframe para los vectores

vectors = data.loc[data['type'].isin(['vector'])]
vectors = vectors.drop(columns=[vectors.columns[1],
                                vectors.columns[4],
                                vectors.columns[5]])

#almacenamos las iteraciones en una lista de sub-dataframes

dataframes = []
for run, group in vectors.groupby('run'):
  dataframes.append(group)

for j, df in enumerate(dataframes):
  df['run'] = (df.iat[0, 0])[8]
  df.reset_index(drop=True, inplace=True)


print("Numero de iteraciones a analizar: ", len(dataframes))


# iterations  0, 1, 2, 3, 4 = 0.1, 0.2, 0.3, 0.5, 0.1


dataframes[5]
#dataframes[1].loc[dataframes[1]['name'] == 'packetsDropped'].index


In [ ]:
dataframes[0]

In [ ]:
def parse_vector(df, index):
  time = df['vectime'].loc[[index]].tolist()
  value = df['vecvalue'].loc[[index]].tolist()
  return time, value

def parse_value(val):
  res = list(map(float, val[0].split()))
  return res

array = []

for n, df in enumerate(dataframes):
  # Omnet guarda los valores de nuestro "vector" (lista de valores) en una celda,
  # por lo que tengo que esxtraer estos valores
  time_genpkt, count_genpkt = parse_vector(df, 0)   #Paquetes generados
  time_tratx, buffer_tratx = parse_vector(df, 1)    #Buffer TraTx
  time_sentpkt, count_sentpkt = parse_vector(df,2)  #Paquetes enviados por TraTx
  time_q0, buffer_q0 = parse_vector(df, 3)          #Buffer general 0
  time_trarx, buffer_trarx = parse_vector(df, 4)    #Buffer TraRx
  time_rcv, count_rcv = parse_vector(df, 5)         #Paquetes recibidos TraRx
  time_ctrlpkt, count_ctrlpkt = parse_vector(df, 6) #Control Packets enviados
  time_q1, buffer_q1 = parse_vector(df, 7)          #Buffer general 1
  time_ctrlrcv, count_ctrlrcv = parse_vector(df, 8) #Control Packets Recieved
  time_delay, count_delay = parse_vector(df, 9)     #Sink Delay
  time_pktrcv, count_pktrcv = parse_vector(df, 10)  #Paquetes entregados a sink

  # Los valores que sacamos estan en un string separados por espacios,
  # los convertimos a listas para poder graficarlos

  parsed_data = []  
  # parsed_data.append((parse_value(), parse_value()))
  parsed_data.append((parse_value(time_genpkt), parse_value(count_genpkt)))
  parsed_data.append((parse_value(time_tratx), parse_value(buffer_tratx)))
  parsed_data.append((parse_value(time_sentpkt), parse_value(count_sentpkt)))
  parsed_data.append((parse_value(time_q0), parse_value(buffer_q0)))
  parsed_data.append((parse_value(time_trarx), parse_value(buffer_trarx)))
  parsed_data.append((parse_value(time_rcv), parse_value(count_rcv)))
  parsed_data.append((parse_value(time_ctrlpkt), parse_value(count_ctrlpkt)))
  parsed_data.append((parse_value(time_q1), parse_value(buffer_q1)))
  parsed_data.append((parse_value(time_ctrlrcv), parse_value(count_ctrlrcv)))
  parsed_data.append((parse_value(time_delay), parse_value(count_delay)))
  parsed_data.append((parse_value(time_pktrcv), parse_value(count_pktrcv)))
  if 'packetsDropped' in df['name'].values:
    time_drops, count_drops = parse_vector(df, 11)
    parsed_data.append((parse_value(time_drops), parse_value(count_drops)))
  else:
    parsed_data.append(([],[]))
  array.append(parsed_data)

#array[Iteracion][Vector][Time/Value]

#print(array[0][11][1])

## Graficos

In [ ]:
# Define el numero de subplots segun el length de nuestro array
num_subplots = len(array)

# Calcula el numero de filas y columnas para los subplots
num_rows = (num_subplots + 2) // 3  # Redondeamos
num_cols = min(num_subplots, 3)

# Crea la figura con subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 4*num_rows))

if num_subplots == 1:
    axs = np.array([axs])

# Itera el array y crea cada subplot
for n, i in enumerate(array):
    row = n // num_cols
    col = n % num_cols

    ax = axs[row, col]  # Selecciona el subplot

    ax.plot(i[1][0], i[1][1], color='tab:blue', label="Tx")
    ax.plot(i[3][0], i[3][1], color='tab:orange', label="Nx")
    ax.plot(i[4][0], i[4][1], color='tab:green', label="Rx")

    ax.set_title('Case Study 2, generationInterval: %s' % interval[n], fontsize=9)
    ax.set_xlabel("tiempo de simulacion")
    ax.set_ylabel("Cantidad de paquetes en el buffer")
    ax.legend()

# Remueve cualquier subplot vacia
if num_subplots < num_rows * num_cols:
    for i in range(num_subplots, num_rows * num_cols):
        fig.delaxes(axs.flatten()[i])

# Titulo principal
plt.suptitle('Ocupacion de buffers en el sistema', fontsize=12)

# Ajusta el espaciado entre subplots
plt.tight_layout()

# Muestra el plot
plt.show()


In [ ]:
# Define el numero de subplots segun el length de nuestro array
num_subplots = len(array)

# Calcula el numero de filas y columnas para los subplots
num_rows = (num_subplots + 2) // 3  # Round up to the nearest integer
num_cols = min(num_subplots, 3)

# Crea la figura con subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
fig.suptitle('Paquetes', fontsize=16)

# Itera el array y crea cada subplot
for n, i in enumerate(array):
    row = n // num_cols
    col = n % num_cols

    ax = axs[row, col]  # Selecciona el subplot

    ax.set_title('Case Study 1, generationInterval: %s' % interval[n])

    colors = ['royalblue', 'dodgerblue', 'limegreen',
              'darkgreen', 'orange', 'darkorange', 'red']  #Colores para las barras
      
    bars = ax.bar(['Generated', 'Sent Tx', 'Rcvd Rx',
                  'Rcvd Sink', 'Sent Control', 'Rcvd Control', 'Dropped'], 
                    [len(i[0][1]),len(i[2][1]), len(i[5][1]),
                    len(i[10][1]), len(i[6][1]), len(i[8][1]), len(i[11][1])],
                    width=0.7, color=colors)
    
   
    ax.text(.99, .99,
            'Buffers en t=200: Tx:%i Nx:%i Rx:%i' %(i[1][1][len(i[1][1])-1],
                                           i[3][1][len(i[3][1])-1],
                                           i[4][1][len(i[4][1])-1]),
            style='italic',
            horizontalalignment='right',
            verticalalignment='top',
            transform=ax.transAxes)

    for bar in bars:
      height = bar.get_height()
      ax.text(bar.get_x() + bar.get_width() / 2, height, '%d' % int(height),
              ha='center', va='bottom')    
    # Rota las etiquetas en diagonal
    ax.set_xticks(range(len(['Generated Tx', 'Sent Tx', 'Rcvd Rx', 'Rcvd Sink',
                             'Sent Control', 'Rcvd Control', 'Dropped'])))
    ax.set_xticklabels(['Generated Tx', 'Sent Tx', 'Rcvd Rx', 'Rcvd Sink',
                             'Sent Control', 'Rcvd Control', 'Dropped'],
                      rotation=45, ha='right', va='top')

# Remover cualquier subplot vacia
if num_subplots < num_rows * num_cols:
    for i in range(num_subplots, num_rows * num_cols):
        fig.delaxes(axs.flatten()[i])



# Ajustar el espaciado entre subplots
fig.tight_layout()

# Mostrar el plot
plt.show()

# Grafico de cargas

In [ ]:
#parse info para grafico de carga util y recibida

totalsent = []
totalrcv =[]

for i in array:
#  pass
#  sent = len(i[3][1])  #analisis
#  rcv = len(i[4][1])   #analisis
  sent = len(i[2][1])  #diseño
  rcv = len(i[5][1])   #diseño
  totalsent.append(sent)
  totalrcv.append(rcv)

print(totalsent)
print(totalrcv)

'''
intervals = [0.1, 0.2, 0.3, 0.5, 0.7, 1]
sent_analisis_case1 = [1979, 1001, 694, 397, 286, 191]
rcv_analisis_case1 =  [998, 987, 693, 396, 285, 191]
sent_analisis_case2 = [1979, 1001, 694, 397, 286, 191]
rcv_analisis_case2 =  [998, 987, 693, 396, 285, 191]
sent_design_case1 =   [1148, 999, 694, 397, 286, 191]
rcv_design_case1 =    [1147, 997, 694, 397, 285, 191]
sent_design_case2 =   [1971, 999, 694, 397, 286, 191]
rcv_design_case2 =    [999, 987, 693, 397, 285, 191]

intervals = [0.1, 0.07, 0.05, 0.03, 0.02, 0.01]
analisis_sent1 = [1979, 2878, 4112, 6643, 9878, 19948]
analisis_rcv1 =  [998, 997, 998, 998, 998, 998]
analisis_sent2 = [1979, 2878, 4112, 6643, 9878, 19948]
analisis_rcv2 =  [998, 997, 998, 998, 998, 998]
design_sent1 = [1979, 2878, 4112, 6643, 9878, 19948]
design_rcv1 = [1147, 1164, 1165, 1165, 1165, 1165]
design_sent2 = [1979, 2878, 4112, 6643, 9878, 19948]
design_rcv2  = [999, 998, 999, 999, 999, 999]
'''


intervals = [1, 0.7, 0.5, 0.3, 0.2, 0.1, 0.07, 0.05, 0.03, 0.02, 0.01]

analisis_sent1 = [191, 286, 397, 694, 1001, 1979, 2878, 4112, 6643, 9878, 19948]
analisis_rcv1 =  [191, 285, 396, 693, 987, 998, 997, 998, 998, 998, 998]

analisis_sent2 = [191, 286, 397, 694, 1001, 1979, 2878, 4112, 6643, 9878, 19948]
analisis_rcv2 =  [191, 285, 396, 693, 987, 998, 997, 998, 998, 998, 998]

design_sent1 = [191, 286, 397, 694, 999, 1148, 1165, 1166, 1166, 1166, 1166]  #oknigga
design_rcv1 =  [191, 285, 397, 694, 997, 1147, 1164, 1165, 1165, 1165, 1165]

design_sent2 = [191, 286, 397, 694, 999, 1971, 1995, 2000, 2000, 2000, 2000] #okdou
design_rcv2  = [191, 285, 397, 693, 987, 999, 998, 999, 999, 999, 999]

todas = [analisis_sent1, analisis_sent2, design_sent1, design_sent2,
         analisis_rcv1, analisis_rcv2, design_rcv1, design_rcv2]

ofrecida = [analisis_sent1, analisis_sent2, design_sent1, design_sent2]
util = [analisis_rcv1, analisis_rcv2, design_rcv1, design_rcv2]

for lst in todas:
    for i in range(len(lst)):
        lst[i] = lst[i] / 200
      
#print(todas)
print('sent design', design_sent1)
print('rcv design', design_rcv1)
print('rcv design', design_rcv2)
print('sent analisis:', analisis_sent1)
print('rcv analisis: ', analisis_rcv1)

In [ ]:
intervals = [1, 0.7, 0.5, 0.3, 0.2, 0.1, 0.07, 0.05, 0.03, 0.02, 0.01]

ofrecida = [analisis_sent1, analisis_sent2, design_sent1, design_sent2]
util = [analisis_rcv1, analisis_rcv2, design_rcv1, design_rcv2]

fig, axs = plt.subplots(2, 2, figsize=(15, 10))

colors = ['b', 'g', 'r', 'c'] 

titles = ['Analisis, caso 1', 'Analisis, caso 2', 'Diseño, caso 1', 'Diseño, caso 2']

for i in range(len(ofrecida)):
    row = i // 2 
    col = i % 2  

    axs[row, col].scatter(ofrecida[i], util[i], c=colors[i], label=f'generationInterval')
    axs[row, col].plot(ofrecida[i], util[i], linestyle='-', color='gray', alpha=0.7)

    for j, txt in enumerate(intervals):
        axs[row, col].annotate(txt, (ofrecida[i][j], util[i][j]), textcoords="offset points", xytext=(0, 10),
                        ha='center', fontsize=8)

    axs[row, col].set_xlabel('Carga Ofrecida (paquetes/seg)')
    axs[row, col].set_ylabel('Carga Util (paquetes/seg)')
    axs[row, col].set_title(titles[i])
    axs[row, col].legend()
    axs[row, col].grid(True)

plt.tight_layout()
plt.show()
